In [3]:
import json
import pandas as pd
import numpy as np
from pprint import pprint

In [4]:
file_dir= "/Users/bnb/Desktop/analytics/8_ETL/Movies_ETL/resources/"

In [5]:
with open(f'{file_dir}wikipedia-movies.json', mode= 'r') as file:
    wiki_movies_raw= json.load(file)

In [6]:
len(wiki_movies_raw)

7311

In [7]:
kaggle_metadata= pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings= pd.read_csv(f'{file_dir}ratings.csv')

In [8]:
kaggle_metadata.sample(n=5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
33695,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,29977,tt1471236,en,Pou hark wong,Although Randy's family is involved in the cas...,...,2009-11-22,0.0,122.0,"[{'iso_639_1': 'zh', 'name': '普通话'}]",Released,NaN,Poker King,False,5.8,6.0
18352,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",http://www.joyfulnoisethemovie.com,63574,tt1710396,en,Joyful Noise,G.G. Sparrow faces off with her choir's newly ...,...,2012-01-13,0.0,117.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Dream a whole lot louder,Joyful Noise,False,6.7,33.0
28198,False,NaN,0,"[{'id': 10751, 'name': 'Family'}]",NaN,30583,tt0100264,sv,Nils Karlsson Pyssling,Nils Karlsson Pyssling is a 1990 Swedish famil...,...,1990-11-09,0.0,75.0,"[{'iso_639_1': 'sv', 'name': 'svenska'}]",Released,NaN,Nils Karlsson Pyssling,False,5.8,5.0
3761,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,25128,tt0234137,en,Love & Sex,"When her rather explicit copy is rejected, mag...",...,2000-01-24,0.0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Love & Sex,False,5.7,21.0
10255,False,NaN,7500000,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",http://marvel.com/movies/movie/4/man-thing,18882,tt0290747,en,Man-Thing,Agents of an oil tycoon vanish while exploring...,...,2005-04-21,143000.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Nature of Fear,Man-Thing,False,4.0,38.0


In [9]:
ratings.sample(n=5)

,userId,movieId,rating,timestamp
9627725,99291,2325,2.0,1111988544
8161896,84145,1234,5.0,1137378026
22807554,236983,2594,4.0,1271012257
1645886,17088,588,4.0,962333823
2313209,24025,4661,4.0,1459091595


In [15]:
wiki_movies_df= pd.DataFrame(wiki_movies_raw)
len(wiki_movies_df)

7311

In [11]:
wiki_movies= [movie for movie in wiki_movies_raw
            if ('Director' in movie or 'Directed by' in movie)
            and 'imdb_link' in movie
            and 'No. of episodes' not in movie]
len(wiki_movies)

7076

In [12]:
def clean_movie(movie):
    movie= dict(movie) #create non-destructive copy
    return movie

In [16]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

7060    https://en.wikipedia.org/wiki/The_Insult_(film)
7293     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [17]:
sorted(wiki_movies_df.columns.to_list())

['Actor control',
 'Adaptation by',
 'Alias',
 'Alma mater',
 'Also known as',
 'Animation by',
 'Arabic',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Characters',
 'Children',
 'Chinese',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Country of origin',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Divisions',
 'Duration',
 'Edited by',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'French',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Ha

In [18]:
def clean_movie(movie):
    movie= dict(movie) #create non-destructive copy
    alt_titles= {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key]= movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles']= alt_titles
    return movie

In [19]:
clean_movies= [clean_movie(movie) for movie in wiki_movies]

In [20]:
wiki_movies_df= pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']